## Морфологическая  дизамбигуация

Неоднозначность - одно из тех, свойств языка, которые делают его трудным (как для человеков так и для компьютеров.)

Неоднозначность проявляется на разных уровнях языка. И под каждую есть своя задача в NLP.  
Морфологическая неоднозначность - это когда одна и та же форма слова может иметь несколько вариантов морфологического описания.  
Например, ``стали`` - может быть глаголом в прошедшем времени мн.ч 3.л (``они стали``), а может - существительным женского рода в родительном падеже (``коробка из стали``).

Скорее всего, вы уже знаете или догадываетесь, что неоднозначность снимается в контексте.   
Однако контекст это не всегда несколько слов по соседству (как в примерах выше).   
Иногда это контекст находится в других, необязательно соседних предложениях.   
Например, предложение: ``Эти типы стали есть на складе.`` многозначно без другого предложения, в котором говорится о чём речь (о стали, или о типах).

Поэтому в теории - это очень сложная задача. И над ней работают многие комп. лингвисты.

Однако на практике эта задача либо вообще не стоит, либо решается достаточно хорошо.

Давайте посмотрим, почему:

Для русского есть готовые инструменты - pymorphy и mystem. И тот и другой умеют выдавать грамматическую информацию.

In [0]:
from lxml import etree
!pip install pymorphy2[fast]
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import classification_report
import numpy as np
from collections import Counter

     |████████████████████████████████| 51kB 2.1MB/s 
     |████████████████████████████████| 7.1MB 7.7MB/s 
     |████████████████████████████████| 256kB 54.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/d4/88/d0/4e4abc83eb8f59a71e8dbd8ba99fd5615a3af1fac1ef7f8825
Successfully built DAWG


Чтобы оценить как они справляются с неоднозначностью нам нужен размеченный корпус. А точнее корпус-снятник (т.е. тот в котором вручную разрешена неоднозначность). Обычно для этого используют НКРЯ, но там нужно запрашивать и подписывать какое-то соглашение. Поэтому мы возьмем OpenCorpora, который можно скачать без этих сложностей вот тут - http://opencorpora.org/?page=downloads (нужен снятник без UNK).

Сам корпус в xml. Для того, чтобы достать все в питоновские структуры данных, удобно использовать lxml и xpath.

In [0]:
open_corpora = etree.fromstring(open('annot.opcorpora.no_ambig_strict.xml', 'rb').read())

Так достанутся все предложения.

In [0]:
# document root + all tokens
sentences = open_corpora.xpath('//tokens')

In [0]:
import xml.dom.minidom

dom = xml.dom.minidom.parse('annot.opcorpora.no_ambig_strict.xml')
pretty_xml_as_string = dom.toprettyxml()

In [0]:
print(pretty_xml_as_string[750:3000])

agraphs>
			
      
			<paragraph id="1">
				
        
				<sentence id="1">
					
          
					<source>«Школа злословия» учит прикусить язык</source>
					
          
					<tokens>
						
            
						<token id="1" text="«">
							<tfr rev_id="2420236" t="«">
								<v>
									<l id="0" t="«">
										<g v="PNCT"/>
									</l>
								</v>
							</tfr>
						</token>
						
            
						<token id="2" text="Школа">
							<tfr rev_id="834910" t="Школа">
								<v>
									<l id="380220" t="школа">
										<g v="NOUN"/>
										<g v="inan"/>
										<g v="femn"/>
										<g v="sing"/>
										<g v="nomn"/>
									</l>
								</v>
							</tfr>
						</token>
						
            
						<token id="3" text="злословия">
							<tfr rev_id="2632816" t="злословия">
								<v>
									<l id="115766" t="злословие">
										<g v="NOUN"/>
										<g v="inan"/>
										<g v="neut"/>
										<g v="sing"/>
										<g v="gent"/>
									</l>
							

А так в отдельном предложении достанутся все слова.

In [0]:
# get all tokens
tokens = sentences[0].xpath('token')

Для токена форма слова достается вот так:

In [0]:
# get attribute value
tokens[0].xpath('@text')

['«']

А грамматическая информация вот так:

In [0]:
# full path or just all @v attributes
tokens[1].xpath('tfr/v/l/g/@v'), tokens[1].xpath('.//@v')

(['NOUN', 'inan', 'femn', 'sing', 'nomn'],
 ['NOUN', 'inan', 'femn', 'sing', 'nomn'])

Соберем весь корпус в список. Для начала будем смотреть только на часть речи.

In [0]:
corpus = []

for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        sent_tagged.append((word[0], gram_info[0])) # word and POS
    corpus.append(sent_tagged)

In [0]:
len(corpus)

10597

In [0]:
corpus[0]

[('«', 'PNCT'),
 ('Школа', 'NOUN'),
 ('злословия', 'NOUN'),
 ('»', 'PNCT'),
 ('учит', 'VERB'),
 ('прикусить', 'INFN'),
 ('язык', 'NOUN')]

Воспользуемся pymorphy.

In [0]:
morph = MorphAnalyzer()

In [0]:
morph.parse('слово')[0].tag.POS

'NOUN'

Теперь просто пройдемся по каждому слову, предскажем его часть речи через пайморфи и сравним с тем, что стоит в корпусе. Если совпадает добавим в список 1, если нет 0. Усреднив нули и единицы получим accuracy.

In [0]:
preds = []
mistakes = Counter()

for sent in corpus:
    for word, tag in sent:
        pred = str(morph.parse(word)[0].tag).split(',')[0].split(' ')[0] # in order to get PNCT as POS in case of punctuation
        preds.append(int(pred == tag))
        if not preds[-1]:
            mistakes.update([(word, tag, pred)])

Видно, что для части речи проблема неоднозначности особо и незаметна.

In [0]:
print(np.mean(preds))

0.9829557744163578


А если посмотреть на ошибки, то видно, что они происходят в каких-то не очень значимых случаях.

In [0]:
 mistakes.most_common(5)

[(('также', 'PRCL', 'CONJ'), 90),
 (('тоже', 'PRCL', 'ADVB'), 37),
 (('этом', 'ADJF', 'NPRO'), 36),
 (('Также', 'PRCL', 'CONJ'), 24),
 (('=', 'SYMB', 'UNKN'), 20)]

Попробуем теперь предсказывать сразу всю грамматическую информацию.

In [0]:
corpus = []

for sentence in open_corpora.xpath('//tokens'):
    sent_tagged = []
    for token in sentence.xpath('token'):
        word = token.xpath('@text')
        gram_info = token.xpath('tfr/v/l/g/@v')
        sent_tagged.append((word[0], set(gram_info)))

    corpus.append(sent_tagged)

In [0]:
preds = []
mistakes = Counter()

for sent in corpus:
    for word, tag in sent:
        pred = set(str(morph.parse(word)[0].tag).replace(' ', ',').split(','))
        preds.append(len(pred & tag) / len(pred | tag))
        if preds[-1] < 0.5:
            mistakes.update([(word, tuple(tag), tuple(pred))])

Оценивание правда придется поменять. Так как тэгов несколько и они могут быть в разном порядке мы не можем просто их склеить. Поэтому посчитаем меру жаккара между множествами тэгов.

In [0]:
np.mean(preds), np.std(preds)

(0.9335980953841342, 0.18528548605606146)

Она достаточно высокая.

А ошибки все те же.

In [0]:
mistakes.most_common(10)

[(('также', ('PRCL',), ('CONJ',)), 90),
 (('тоже', ('PRCL',), ('ADVB',)), 37),
 (('человек',
   ('gent', 'masc', 'NOUN', 'anim', 'plur'),
   ('sing', 'nomn', 'masc', 'NOUN', 'anim')),
  34),
 (('этом',
   ('loct', 'sing', 'Subx', 'ADJF', 'masc', 'Anph', 'Apro'),
   ('loct', 'neut', 'NPRO', 'sing')),
  27),
 (('Ссылки',
   ('femn', 'nomn', 'inan', 'NOUN', 'plur'),
   ('femn', 'sing', 'gent', 'inan', 'NOUN')),
  26),
 (('Также', ('PRCL',), ('CONJ',)), 24),
 (('Примечания',
   ('nomn', 'inan', 'neut', 'NOUN', 'plur'),
   ('sing', 'gent', 'inan', 'neut', 'NOUN')),
  23),
 (('=', ('SYMB',), ('UNKN',)), 20),
 (('№', ('SYMB',), ('UNKN',)), 19),
 (('>', ('SYMB',), ('UNKN',)), 19)]

Поэтому на практике, можно забить на неоднозначность.

Если все таки нужно (или хочется) разрешить неоднозначность - можно использовать mystem (там есть дизамбигуация). Но там своя токенизация и сложно будет оценивать качество на уже токенизированном корпусе.

Либо воспользоваться готовыми иструментами и обучить свой сниматель неоднозначности...

Про это лучше рассказать в колабе - https://colab.research.google.com/drive/1uTLlHbYdh8XA2Pbe7YAivS82FciLjU1b